# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [4]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [19]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [20]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [21]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [22]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [23]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [24]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Based on the information provided, here are some exercises that can help alleviate lower back pain:\n\n1. Cat-Cow Stretch:\n- Start on your hands and knees.\n- Alternate between arching your back up (cat position) and letting it sag down (cow position).\n- Repeat for 10-15 repetitions.\n\n2. Bird Dog:\n- Begin on your hands and knees.\n- Extend opposite arm and leg simultaneously while keeping your core engaged.\n- Hold each extension for about 5 seconds.\n- Switch sides and repeat for 10 repetitions per side.\n\n3. Pelvic Tilts:\n- Lie on your back with knees bent.\n- Flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis upward.\n- Hold for 10 seconds.\n- Repeat 8-12 times.\n\nThese exercises are gentle and aimed at stretching and strengthening the lower back muscles, which can help reduce pain and improve mobility. Remember to perform these exercises slowly and consult with a healthcare professional if you have any concerns or specific medi

In [18]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. Adequate and quality sleep—typically 7-9 hours per night—supports physical health by allowing the body to repair tissues, regulate hormones, and strengthen the immune system. It also enhances mental well-being, cognitive functions such as memory and learning, and mood stability. Poor sleep or sleep disorders like insomnia can negatively affect these processes, leading to increased health risks, weakened immunity, and mental health issues. Maintaining good sleep hygiene and creating an optimal sleep environment are essential strategies for promoting overall health.'

In [19]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water and staying hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using peppermint or lavender essential oils\n- Maintaining a regular sleep schedule\n- Practicing deep breathing, progressive muscle relaxation, or grounding techniques\n- Taking short walks in nature\n- Listening to calming music\n\nThese approaches can help alleviate stress and headache symptoms naturally.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [25]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [26]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [27]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n- **Bird Dog:** From your hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your lower back against the floor by tightening your abdominal muscles and tilting your pelvis slightly upward. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfort and prevent future episodes.'

In [12]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health. Good sleep hygiene and a consistent sleep schedule help ensure quality sleep, which is essential for physical and mental well-being. Adequate sleep supports the immune system, improves mood, boosts energy levels, enhances cognitive function, and promotes better digestive health. Creating an optimal sleep environment—such as maintaining a cool, dark, quiet bedroom and avoiding stimulating activities before bed—can significantly improve sleep quality. Poor or insufficient sleep can lead to health issues like impaired immunity, mental health challenges, and increased risk of chronic conditions.'

In [24]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include practicing relaxation techniques such as deep breathing and meditation. For headaches, staying well-hydrated, managing stress through relaxation, ensuring adequate sleep, and avoiding known triggers like certain foods or eye strain can help reduce their frequency and intensity. Herbal teas like chamomile or valerian root may also promote relaxation and help alleviate headaches related to stress.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:

Better where we want to do specific and exact keyword matching and less bothered by semantic meaning. 
For example if searching for a product via its name or a product code (like ASIN on Amazon.com) then BM25 is likely to be better, since we are uninterested in semantically similar product codes. 
Same goes when looking up a programming error code like a 400 error etc. Really the ony interest at this particular time is in that exact error not other semantically similar error:

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [28]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [29]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [30]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"To help with lower back pain, some effective exercises include:\n\n- **Cat-Cow Stretch**: Begin on hands and knees, then alternate between arching your back up (cat) and letting it sag down (cow). Aim for 10-15 repetitions.\n- **Bird Dog**: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds and switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts**: Lie on your back with knees bent, tighten your abdominal muscles, and tilt your pelvis upward to flatten your lower back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle exercises can help alleviate discomfort and prevent future episodes. However, it's always best to consult with a healthcare professional before starting new exercise routines."

In [31]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly impacts overall health by supporting physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep, typically 7-9 hours for adults, in a proper environment helps maintain these processes, contributing to better physical health, mental clarity, and emotional stability. Conversely, poor sleep or sleep disorders like insomnia can negatively affect overall health and well-being.'

In [30]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include drinking water to stay hydrated, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gently massaging the temples and neck, and using essential oils like peppermint or lavender. Additionally, practices such as deep breathing, progressive muscle relaxation, grounding techniques, taking short walks in nature, and listening to calming music can help alleviate stress.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [32]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [33]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [34]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Perform 10 repetitions per side.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds. Repeat 8-12 times.\n\nThese exercises, performed gently and regularly, can help alleviate and prevent lower

In [35]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical well-being, mental health, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep (typically 7-9 hours for adults) supports the immune system, reduces stress, and promotes emotional stability. Poor sleep or sleep disturbances like insomnia can lead to physical health issues such as headaches, fatigue, and digestive problems, as well as mental health challenges like increased stress and mood disturbances. Therefore, maintaining good sleep hygiene and ensuring quality sleep are vital for overall health and wellness.'

In [35]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises (inhale for 4 counts, hold for 4, exhale for 4)\n- Progressive muscle relaxation (tensing and releasing muscle groups)\n- Grounding techniques (naming things you see, hear, feel, smell, and taste)\n- Taking short walks, preferably in nature\n- Listening to calming music\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of temples and neck\n- Using essential oils like peppermint or lavender\n\nThese methods can help provide immediate relief and promote overall relaxation.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

It can be hard to articulate a query in a way that makes it crystal-clear for a search algorithm to interpret the way the user intended. Missing assumptions, ambiguities, and we are in the main (at least until recently) pre-conditioned to providing keywords for searching, not longer form prompts. So reformulating the query if a way of expanding the search space in the hope of returning potentially strong candidate results.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [36]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [37]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [38]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [39]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [40]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [41]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'To help with lower back pain, the recommended exercises include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfor

In [42]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly affects overall health in several ways. It is crucial for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adults generally need 7-9 hours of sleep per night. Good sleep quality supports healthy immune function, maintains mood stability, and enhances concentration and learning. Poor sleep or sleep disturbances can lead to increased risk of health issues such as fatigue, headaches, impaired cognitive function, and emotional stress. Therefore, maintaining healthy sleep habits and hygiene is vital for overall wellness.'

In [43]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing, engaging in progressive muscle relaxation, doing grounding exercises, taking short walks especially in nature, listening to calming music, and practicing mindfulness or meditation. For headaches specifically, natural remedies include drinking plenty of water to stay hydrated, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gently massaging the temples and neck, using essential oils like peppermint or lavender, and maintaining a regular sleep schedule.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [42]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [43]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [44]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch**: Start on your hands and knees, then alternate between arching your back up (like a cat) and letting it sag down (like a cow). Perform 10-15 repetitions.\n\n- **Bird Dog**: From hands and knees, extend opposite arm and leg simultaneously while keeping your core engaged. Hold each extension for about 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Pelvic Tilts**: Lie on your back with knees bent. Flatten your lower back against the floor by tightening your abdominal muscles and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\n- **Partial Crunches**: Lie on your back with knees bent, cross your arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor briefly. Then lower back down. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch**: Lie on your back and pull one knee toward your chest, keeping the other foot flat on the floor. Hol

In [47]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical repair, mental well-being, and cognitive functions such as memory and learning. During sleep, tissues are repaired, hormones that regulate growth and appetite are released, and the brain consolidates memories. Adequate sleep (7-9 hours per night) supports immune function, reduces stress, and boosts mental health. Poor sleep or sleep disturbances like insomnia can negatively affect physical health, mental well-being, and increase vulnerability to illness. Maintaining good sleep hygiene—such as keeping a consistent schedule, creating a comfortable sleep environment, and practicing relaxation techniques—can help promote better sleep quality and, consequently, improve overall health.'

In [48]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include deep breathing exercises, progressive muscle relaxation, grounding techniques (such as naming things you see, hear, feel, smell, and taste), taking short walks especially in nature, and listening to calming music. \n\nFor headaches, natural remedies include staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark and quiet room, gentle massage of the temples and neck, using essential oils such as peppermint or lavender, maintaining a regular sleep schedule, and using caffeine in small amounts if appropriate.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [45]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [46]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [47]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [48]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [49]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [50]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help alleviate lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help relieve lower back discomfort and may prevent future episodes.'

In [51]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly impacts overall health by supporting physical restoration, mental well-being, and cognitive functions. During sleep, the body repairs tissues, regenerates cells, and releases hormones that regulate growth and appetite. Adequate sleep—typically 7-9 hours for adults—also helps consolidate memories and improve learning. Good sleep quality is crucial for maintaining a healthy immune system, managing stress, and preventing chronic conditions such as hypertension, diabetes, and mental health issues. Conversely, poor sleep can lead to fatigue, impair cognitive function, weaken immunity, and increase the risk of various health problems. Therefore, prioritizing consistent, quality sleep is essential for overall health and wellness.'

In [57]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises: Inhale for 4 counts, hold for 4, exhale for 4, and hold for 4.\n- Progressive muscle relaxation: Tense and relax muscle groups, such as from toes to head.\n- Grounding techniques: Name 5 things you see, 4 you hear, 3 you feel, 2 you smell, and 1 you taste.\n- Taking short walks, preferably in nature.\n- Listening to calming music.\n- Applying peppermint or lavender essential oils.\n- Staying well-hydrated by drinking water.\n- Resting in a dark, quiet room.\n- Gentle massage of temples and neck.\n\nThese approaches can help reduce tension, promote relaxation, and alleviate headache symptoms naturally.'

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

Highly repetative questions with similar sentence structure are likely to be problematic for semantic search because they will have high semantic overlap and it will be harder to discriminate what differentiates each question (which 
is what is needed) rather than what makes them similar. 

BM25 may work better here since its likely to give more weight to terms that are likely to be differentiators rather than common sentence structure words found in FAQS (where/what/how do.....).

Generally a hybrid approach (lexical + semantic search) gives the flexibility of addressing these kinds of questions, plus any other kinds of questions that don't fit this pattern and may be a better fit for semantic search. 

---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [ ]:
### YOUR CODE HERE

![image](retreivers.png)

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))

generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/var/folders/bz/qcwzc_9j6zgggtsc_fbjryy40000gp/T/ipykernel_57514/1536617908.py:5: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
/var/folders/bz/qcwzc_9j6zgggtsc_fbjryy40000gp/T/ipykernel_57514/1536617908.py:7: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


In [12]:
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import (
    SingleHopSpecificQuerySynthesizer,
)
query_distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
# new since 0.4.2
dataset = generator.generate_with_chunks(wellness_docs, testset_size=20, query_distribution=query_distribution)

Applying SummaryExtractor:   0%|          | 0/45 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/45 [00:00<?, ?it/s]

Node 008b1a11-ea96-4a03-92e7-3208ed98f988 does not have a summary. Skipping filtering.
Node 92213be8-9f63-4a2a-ba19-5b61231d823c does not have a summary. Skipping filtering.
Node e829393e-c301-4d89-882c-fa70db2478e5 does not have a summary. Skipping filtering.
Node b59f4f31-092f-4e2e-b67a-8bf728bb6b12 does not have a summary. Skipping filtering.
Node 8a3e52c6-6068-490c-8376-19e338ba99c6 does not have a summary. Skipping filtering.
Node 597af84d-8268-4245-affa-57f5c0f340d0 does not have a summary. Skipping filtering.
Node 48febde2-a429-4585-bd9e-33835f1b3b7e does not have a summary. Skipping filtering.
Node c9ca0f72-b214-47b9-83b5-2684db6b513a does not have a summary. Skipping filtering.
Node bb6c46c7-c506-4df5-aee7-4b89cfa7705c does not have a summary. Skipping filtering.
Node b18808b3-495c-4389-8e40-0b045af62cd8 does not have a summary. Skipping filtering.
Node d5a3b4e6-eabe-4563-8649-2c870269c7e1 does not have a summary. Skipping filtering.
Node fc69f427-972d-4540-8ded-428283acfb4b d

Applying EmbeddingExtractor:   0%|          | 0/45 [00:00<?, ?it/s]

Applying ThemesExtractor:   0%|          | 0/45 [00:00<?, ?it/s]

...

Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/20 [00:00<?, ?it/s]

In [129]:
dataset.to_pandas().groupby('query_style')['user_input'].count()

query_style
MISSPELLED         5
PERFECT_GRAMMAR    5
POOR_GRAMMAR       5
WEB_SEARCH_LIKE    5
Name: user_input, dtype: int64

In [130]:
dataset.to_pandas().groupby('query_length')['user_input'].count()

query_length
LONG      8
MEDIUM    9
SHORT     3
Name: user_input, dtype: int64

In [14]:
dataset.to_csv(path='eval_dataset.csv')

In [ ]:
# set up  metrics and  langsmith experiments 
from langsmith import Client
import uuid

client = Client()

dataset_name = f"Assignment11 Synthetic Data - AIE9 - {uuid.uuid4()}"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Advanced Retrievers"
)



In [16]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [110]:
from ragas.metrics import ContextPrecision, LLMContextRecall, ContextEntityRecall
from ragas import evaluate, RunConfig, EvaluationDataset
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

/var/folders/bz/qcwzc_9j6zgggtsc_fbjryy40000gp/T/ipykernel_57514/3723646950.py:1: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision, LLMContextRecall, ContextEntityRecall
/var/folders/bz/qcwzc_9j6zgggtsc_fbjryy40000gp/T/ipykernel_57514/3723646950.py:1: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import ContextPrecision, LLMContextRecall, ContextEntityRecall
/var/folders/bz/qcwzc_9j6zgggtsc_fbjryy40000gp/T/ipykernel_57514/3723646950.py:1: DeprecationWarning: Importing ContextEntityRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.

In [111]:
def evaluate_retreiver(candidate_retreiver, retriever_name):
  for test_row in dataset:
    response = candidate_retreiver.invoke({"question" : test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"].content
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

  evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

  result = evaluate(
      dataset=evaluation_dataset,
      metrics=[ContextPrecision(), LLMContextRecall(), ContextEntityRecall()],
      llm=evaluator_llm,
      run_config=custom_run_config,
      experiment_name=retriever_name   # name for this run in LangSmith

  )
  return result 

In [112]:
naive_retriever_result = evaluate_retreiver(naive_retrieval_chain, 'naive_retriever')
naive_retriever_result

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_precision': 0.8699, 'context_recall': 0.9875, 'context_entity_recall': 0.3227}

In [113]:
bm25_retriever_result = evaluate_retreiver(bm25_retrieval_chain, 'bm25_retriever')
bm25_retriever_result


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_precision': 0.4153, 'context_recall': 0.5125, 'context_entity_recall': 0.1313}

In [114]:
contextual_compression_retrieval_result = evaluate_retreiver(contextual_compression_retrieval_chain, 'contextual_compression_retriever')
contextual_compression_retrieval_result

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_precision': 0.9667, 'context_recall': 0.9750, 'context_entity_recall': 0.3718}

In [115]:
multi_query_retrieval_result = evaluate_retreiver(multi_query_retrieval_chain, 'multi_query_retriever')
multi_query_retrieval_result

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_precision': 0.7869, 'context_recall': 1.0000, 'context_entity_recall': 0.3231}

In [116]:
parent_document_retrieval_result = evaluate_retreiver(parent_document_retrieval_chain, 'parent_document_retriever')

parent_document_retrieval_result


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_precision': 0.8750, 'context_recall': 0.9500, 'context_entity_recall': 0.2748}

In [117]:
ensemble_retrieval_result = evaluate_retreiver(ensemble_retrieval_chain, 'ensemble_retriever')
ensemble_retrieval_result

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_precision': 0.5497, 'context_recall': 0.9875, 'context_entity_recall': 0.2713}

In [ ]:
from pandas import DataFrame

results_df = DataFrame([eval(naive_retriever_result.__repr__()), 
eval(bm25_retriever_result.__repr__()),
eval(contextual_compression_retrieval_result.__repr__()),
eval(multi_query_retrieval_result.__repr__()),
eval(parent_document_retrieval_result.__repr__()),
eval(ensemble_retrieval_result.__repr__()) ], 
index=['naive_retriever', 
'bm25_retriever', 
'contextual_compression_retrieval', 
'multi_query_retrieval', 
'parent_document_retrieval',
'ensemble_retrieval'])


In [121]:
results_df

,context_precision,context_recall,context_entity_recall
naive_retriever,0.8699,0.9875,0.3227
bm25_retriever,0.4153,0.5125,0.1313
contextual_compression_retrieval,0.9667,0.9750,0.3718
multi_query_retrieval,0.7869,1.0000,0.3231
parent_document_retrieval,0.8750,0.9500,0.2748
ensemble_retrieval,0.5497,0.9875,0.2713


In [122]:
results_df.to_csv('scores.csv')

Consider the data that we get from Langsmith to run our dataset of evals on (20 examples) (appreciate this includes the evals and output costs as well, but its more about the relative rather than absolute spend)


![image](lsmith_pd.png)

In [ ]:
import pandas as pd
pd.read_csv('langsmith_retreival_compute.csv')

,Retriever,time,tokens,Cost $
0,naive_retriever,124.19s,"309,910",0.1566
1,bm25,67.22,150900,0.0738
2,contextual compression,57.02s,125600,0.0580
3,multiquery,222.55s,390300,0.1904
4,parent document,56.98s,160100,0.0833
5,emsemble,223.02s,552800,0.2684


# Conclusions
Have created a synthetic dataset of 20 Q & As based on the HealthWellnessGuide, and compared the result of 5 different variations of retreiver against a baseline of "naive" RAG. 


To **evaluate the retriever**, the key metrics are context precision and recall which basically is concerned with whether the retreived chunks contain the information needed to answer the question and **how much** of the retreived chunks are relevant to the question

On this particular dataset:
**BM25** doesn't work well  being comparitively poor on both metrics. We'd expect it to do well on questions whose characteristics (See above) are 'web_search_like' and 'short' in length but less on the other styles (which are the majority)

**Multi-query retreival** brings back the most comprehensive set of chunks required to answer the question (I suppose by reformulating the question asked picks up more nuance) at the expense of some noise - retreives some irrelevant chunks. 

**Contextual compression** seems to have the best trade-off of precision and recall both of which are close to the best. The reranking step adds a little cost (0.2 cents per call). **Parent-document** performs almost as well on p/r metrics and latency cost

**Ensemble** throws everything at the problem but comes off less favorably particularly on precision with too much irrelevant chunks retreived (possibly due to BM25) and of course its computationally and $ expensive 

